In [16]:
import pickle
import numpy as np

import dnnlib.tflib as tflib
import dnnlib

In [17]:
def get_sylegan_networks(path_or_url):
    if dnnlib.util.is_url(path_or_url):
        stream = dnnlib.util.open_url(path_or_url, cache_dir='.stylegan2-cache')
    else:
        stream = open(path_or_url, 'rb')

    tflib.init_tf()
    with stream:
        G, D, Gs = pickle.load(stream, encoding='latin1')

    return G, D, Gs

In [18]:
G, D, Gs = get_sylegan_networks('https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl')

In [19]:
print(*Gs.input_shape[1:])

512


In [20]:
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
rnd = np.random.RandomState()

In [21]:
{var: rnd.randn(*var.shape.as_list()) for var in noise_vars}

{<tf.Variable 'G_synthesis_3/noise0:0' shape=(1, 1, 4, 4) dtype=float32_ref>: array([[[[ 1.86805397,  1.32364884,  1.37143711,  2.08021873],
          [-0.53122251,  1.29436614,  0.65669022,  1.4870249 ],
          [-0.01397711, -1.45860596, -0.77379834,  0.18864731],
          [-1.92978057,  1.22339801, -1.04383873,  0.13000067]]]]),
 <tf.Variable 'G_synthesis_3/noise1:0' shape=(1, 1, 8, 8) dtype=float32_ref>: array([[[[ 0.30256681,  0.04285052, -0.10089565,  0.38975725,
           -1.24180605,  0.23103868, -0.36423795,  0.47693846],
          [ 0.06406745, -0.80131586, -2.67881617,  0.4327757 ,
            0.47691519,  0.8188305 ,  1.48313941,  1.36187528],
          [ 0.88635358,  0.07156356, -0.61184606,  0.55017847,
           -0.71747892, -1.14407853, -0.43062724,  0.45863121],
          [-1.36594454,  0.47109354, -0.52315484, -0.54439429,
            0.00389605,  2.29370082,  1.87975057,  2.02592639],
          [-0.62003254, -0.25335041, -0.46013127,  0.37754763,
            1.5